In [0]:
%sql
CREATE TABLE IF NOT EXISTS ecommerce.silver.fact_orders (
  order_id STRING,
  order_ts TIMESTAMP,
  customer_id STRING,
  product_id STRING,
  quantity INT,
  unit_price DOUBLE,
  line_amount DOUBLE,
  payment_method STRING,
  order_status STRING,
  created_at TIMESTAMP,
  updated_at TIMESTAMP
)
USING DELTA
LOCATION 's3://databricks-ai-bi-project/silver/fact_orders/';


CREATE TABLE IF NOT EXISTS ecommerce.silver.dim_customers (
  customer_id STRING,
  country STRING,
  gender STRING,
  created_at TIMESTAMP,
  updated_at TIMESTAMP
)
USING DELTA
LOCATION 's3://databricks-ai-bi-project/silver/dim_customers/';


CREATE TABLE IF NOT EXISTS ecommerce.silver.dim_products (
  product_id STRING,
  product_name STRING,
  brand_id STRING,
  brand_name STRING,
  category_id STRING,
  category_name STRING,
  price DOUBLE,
  created_at TIMESTAMP,
  updated_at TIMESTAMP
)
USING DELTA
LOCATION 's3://databricks-ai-bi-project/silver/dim_products/';



In [0]:
%sql
MERGE INTO ecommerce.silver.fact_orders tgt
USING (
  SELECT
    order_id,
    order_ts,
    customer_id,
    product_id,
    quantity,
    unit_price,
    line_amount,
    payment_method,
    order_status
  FROM ecommerce.bronze.orders
) AS src
ON tgt.order_id = src.order_id
AND tgt.product_id = src.product_id

WHEN MATCHED THEN UPDATE SET
  tgt.order_ts = src.order_ts,
  tgt.customer_id = src.customer_id,
  tgt.quantity = src.quantity,
  tgt.unit_price = src.unit_price,
  tgt.line_amount = src.line_amount,
  tgt.payment_method = src.payment_method,
  tgt.order_status = src.order_status,
  tgt.updated_at = current_timestamp()

WHEN NOT MATCHED THEN INSERT (
  order_id,
  order_ts,
  customer_id,
  product_id,
  quantity,
  unit_price,
  line_amount,
  payment_method,
  order_status,
  created_at,
  updated_at
) VALUES (
  src.order_id,
  src.order_ts,
  src.customer_id,
  src.product_id,
  src.quantity,
  src.unit_price,
  src.line_amount,
  src.payment_method,
  src.order_status,
  current_timestamp(),
  current_timestamp()
);


-- customers 
MERGE INTO ecommerce.silver.dim_customers AS tgt
USING (
  SELECT
    customer_id,
    country,
    gender,
    CAST(created_at AS TIMESTAMP) AS created_at
  FROM ecommerce.bronze.customers
) AS src
ON tgt.customer_id = src.customer_id

WHEN MATCHED THEN UPDATE SET
  tgt.country = src.country,
  tgt.gender = src.gender,
  tgt.updated_at = current_timestamp()

WHEN NOT MATCHED THEN INSERT (
  customer_id,
  country,
  gender,
  created_at,
  updated_at
) VALUES (
  src.customer_id,
  src.country,
  src.gender,
  src.created_at,
  current_timestamp()
);

-- products 

MERGE INTO ecommerce.silver.dim_products AS tgt
USING (
  SELECT
    p.product_id,
    p.product_name,
    p.brand_id,
    b.brand_name,
    p.category_id,
    c.category_name,
    p.price,
    CAST(p.created_at AS TIMESTAMP) AS created_at
  FROM ecommerce.bronze.products p
  JOIN ecommerce.bronze.brands b
    ON p.brand_id = b.brand_id
  JOIN ecommerce.bronze.categories c
    ON p.category_id = c.category_id
) AS src
ON tgt.product_id = src.product_id

WHEN MATCHED THEN UPDATE SET
  tgt.product_name = src.product_name,
  tgt.brand_id = src.brand_id,
  tgt.brand_name = src.brand_name,
  tgt.category_id = src.category_id,
  tgt.category_name = src.category_name,
  tgt.price = src.price,
  tgt.updated_at = current_timestamp()

WHEN NOT MATCHED THEN INSERT (
  product_id,
  product_name,
  brand_id,
  brand_name,
  category_id,
  category_name,
  price,
  created_at,
  updated_at
) VALUES (
  src.product_id,
  src.product_name,
  src.brand_id,
  src.brand_name,
  src.category_id,
  src.category_name,
  src.price,
  src.created_at,
  current_timestamp()
);


